In [472]:
import xml.etree.ElementTree as ET
import pandas as pd
import datetime as dt
import numpy as np
import os

In [473]:
# Path to the XML file exported form Apple Health app
XML_DATA = "C:/PythonProjects/AppleHealth/input_files/apple_health_export/Export.xml"

In [474]:
# Parse XML file exported from Apple Health app
tree = ET.parse(XML_DATA)
root = tree.getroot()

In [475]:
attributes = []
for child in root:
    attributes.append(child.tag)

attributes = list(set(attributes))
print(attributes)

['Record', 'ActivitySummary', 'ExportDate', 'Workout', 'Me']


In [476]:
attributes.remove('ExportDate')
attributes.remove('Me')

print(attributes)

['Record', 'ActivitySummary', 'Workout']


### Workouts

In [504]:
level_1_list = []
level_2_list = []
level_3_list = []

a_list = list(root.iter('Workout'))
for a in range(len(a_list)):
    b_list = list(a_list[a])
    
    for b in range(len(b_list)):
        level_1_list.append(b_list[b].tag)
        c_list = list(b_list[b])

        for c in range(len(c_list)):
            level_2_list.append(c_list[c].tag)
            d_list = list(c_list[c])

            for d in range(len(d_list)):
                level_3_list.append(d_list[d].tag)

level_1_list = list(set(level_1_list))
level_2_list = list(set(level_2_list))
level_3_list = list(set(level_3_list))

In [505]:
### EMPTY LIST: no further levels to investigate
print(level_1_list)
print(level_2_list)
print(level_3_list)

concat_list = list(set(level_1_list + level_2_list + level_3_list))
print(concat_list)

['MetadataEntry', 'WorkoutRoute', 'WorkoutEvent', 'WorkoutStatistics']
['MetadataEntry', 'FileReference']
[]
['MetadataEntry', 'WorkoutStatistics', 'WorkoutRoute', 'WorkoutEvent', 'FileReference']


In [506]:
index = 0
workouts = root.findall('.//Workout')
total_workouts = len(workouts)

for wo in workouts:
    index = index + 1
    wo.attrib['workout_pk'] = str(index)

    for wo_2 in level_1_list:
        workouts_2 = wo.findall(f'.//{wo_2}')
        for x in workouts_2:
            x.attrib['workout_pk'] = str(index)

            for wo_3 in level_2_list:
                workouts_3 = x.findall(f'.//{wo_3}')
                for y in workouts_3:
                    y.attrib['workout_pk'] = str(index)

tree.write('./output_files/Indexed_XML.xml')

In [507]:
# add Workout to list
concat_list.append('Workout')

In [508]:
tree = ET.parse('./output_files/Indexed_XML.xml')
root = tree.getroot()

In [509]:
df_list = []
for elem in concat_list:
    workouts = root.findall(f'.//{elem}')
    globals()[f"{elem}_list"] = []
    df_list.append(str(f"{elem}_list"))
    for wo in workouts:
        globals()[f"{elem}_list"].append(wo.attrib)

In [510]:
df_list

['MetadataEntry_list',
 'WorkoutStatistics_list',
 'WorkoutRoute_list',
 'WorkoutEvent_list',
 'FileReference_list',
 'Workout_list']

In [511]:
for elem in df_list:
    object = globals()[elem]
    df_name = f"df_{elem}"
    print(f"{df_name}:")
    globals()[df_name] = pd.DataFrame(object)
    globals()[df_name]['workout_pk'] = pd.to_numeric(globals()[df_name]['workout_pk'])
    display(pd.DataFrame(object).head(2))

df_MetadataEntry_list:


,key,value,workout_pk
0,HKWasUserEntered,1,NaN
1,HKMetadataKeyHeartRateMotionContext,0,NaN


df_WorkoutStatistics_list:


,type,startDate,endDate,sum,unit,workout_pk,average,minimum,maximum
0,HKQuantityTypeIdentifierActiveEnergyBurned,2021-12-02 19:38:33 +0200,2021-12-02 19:58:33 +0200,254,kcal,1,NaN,NaN,NaN
1,HKQuantityTypeIdentifierDistanceCycling,2021-12-02 19:38:33 +0200,2021-12-02 19:58:33 +0200,8.09,km,1,NaN,NaN,NaN


df_WorkoutRoute_list:


...

,type,date,duration,durationUnit,workout_pk
0,HKWorkoutEventTypeSegment,2022-03-29 16:56:57 +0200,13.28115280667941,min,72
1,HKWorkoutEventTypeSegment,2022-03-29 16:56:57 +0200,25.37857104341189,min,72


df_FileReference_list:


,path,workout_pk
0,/workout-routes/route_2022-03-29_5.39pm.gpx,72
1,/workout-routes/route_2022-04-03_11.15am.gpx,81


df_Workout_list:


,workoutActivityType,duration,durationUnit,sourceName,sourceVersion,creationDate,startDate,endDate,workout_pk,device
0,HKWorkoutActivityTypeCycling,20,min,Peloton,349778,2022-03-28 20:53:49 +0200,2021-12-02 19:38:33 +0200,2021-12-02 19:58:33 +0200,1,NaN
1,HKWorkoutActivityTypeCycling,20,min,Peloton,349778,2022-03-28 20:53:50 +0200,2021-12-03 07:17:51 +0200,2021-12-03 07:37:51 +0200,2,NaN


#### Transform dfs

##### WorkoutStatistics

In [517]:
df_workoutstatistics_melt = pd.melt(df_WorkoutStatistics_list, 
                                    id_vars=['workout_pk', 
                                             'type', 
                                             'unit'], 
                                    value_vars=['sum', 
                                                'average', 
                                                'minimum', 
                                                'maximum'], 
                                    var_name='metric',
                                    value_name='value').dropna()

# shorter observation names
df_workoutstatistics_melt['type'] = df_workoutstatistics_melt['type'].str.replace('HKQuantityTypeIdentifier', '')
df_workoutstatistics_melt['type'] = df_workoutstatistics_melt['type'].str.replace('HKCategoryTypeIdentifier', '')

list(df_workoutstatistics_melt['metric'].unique())

['sum', 'average', 'minimum', 'maximum']

In [518]:
mapMetrics = {'sum': 'Sum', 
              'average': 'Avg',
              'minimum': 'Min',
              'maximum': 'Max'}

df_workoutstatistics_melt = df_workoutstatistics_melt.replace({'metric': mapMetrics})
df_workoutstatistics_melt['unit'] = df_workoutstatistics_melt['unit'].str.lower()

df_workoutstatistics_melt['type'] = df_workoutstatistics_melt['unit'] + "_" + df_workoutstatistics_melt['type'] + df_workoutstatistics_melt['metric']
df_workoutstatistics_melt_drop = df_workoutstatistics_melt.drop(['unit', 'metric'], axis = 1)

In [519]:
df_workoutstatistics_final = df_workoutstatistics_melt_drop.pivot(index='workout_pk', columns='type', values='value')

##### Workouts

##### Prepare DimWorkout

In [529]:
list(df_Workout_list.columns)

['workoutActivityType',
 'duration',
 'durationUnit',
 'sourceName',
 'sourceVersion',
 'creationDate',
 'startDate',
 'endDate',
 'workout_pk',
 'device',
 'Date']

In [541]:
df_Workouts = df_Workout_list.drop(['device', 'sourceVersion'], axis = 1)

df_Workouts['workout_pk'] = pd.to_numeric(df_Workouts['workout_pk'])
df_Workouts['Date'] = pd.to_datetime(df_Workouts['startDate']).dt.strftime('%Y-%m-%d')
df_Workouts['startDate'] = pd.to_datetime(df_Workouts['startDate'])
df_Workouts['endDate'] = pd.to_datetime(df_Workouts['endDate'])
df_Workouts['workoutActivityType'] = df_Workouts['workoutActivityType'].str.replace('HKWorkoutActivityType', '')

In [542]:
DimWorkouts_prep = pd.merge(df_Workouts, df_workoutstatistics_final, on = 'workout_pk', how = 'left')
DimWorkouts_prep.shape

(571, 35)

In [543]:
DimWorkouts = pd.merge(DimWorkouts_prep, df_FileReference_list, on = 'workout_pk', how='left')

### Records

In [575]:
records_list = [x.attrib for x in root.iter('Record')]
df_records = pd.DataFrame(records_list)

df_records = df_records.drop(['sourceName','sourceVersion', 'device'], axis=1)

# parse date columns
df_records['Date'] = pd.to_datetime(df_records['startDate']).dt.strftime('%Y-%m-%d')
df_records['startTime'] = pd.to_datetime(df_records['startDate']).dt.strftime('%H:%M:%S')
df_records['endTime'] = pd.to_datetime(df_records['endDate']).dt.strftime('%H:%M:%S')

# value is numeric, NaN if fails
df_records['value'] = pd.to_numeric(df_records['value'], errors='coerce')

# shorter observation names
df_records['type'] = df_records['type'].str.replace('HKQuantityTypeIdentifier', '')
df_records['type'] = df_records['type'].str.replace('HKCategoryTypeIdentifier', '')

In [591]:

record_types = list(df_records.type.unique())
print(record_types)

['Height', 'BodyMass', 'HeartRate', 'OxygenSaturation', 'RespiratoryRate', 'StepCount', 'DistanceWalkingRunning', 'BasalEnergyBurned', 'ActiveEnergyBurned', 'FlightsClimbed', 'AppleExerciseTime', 'DistanceCycling', 'DistanceSwimming', 'SwimmingStrokeCount', 'RestingHeartRate', 'VO2Max', 'WalkingHeartRateAverage', 'DistanceDownhillSnowSports', 'EnvironmentalAudioExposure', 'HeadphoneAudioExposure', 'WalkingDoubleSupportPercentage', 'SixMinuteWalkTestDistance', 'AppleStandTime', 'WalkingSpeed', 'WalkingStepLength', 'WalkingAsymmetryPercentage', 'StairAscentSpeed', 'StairDescentSpeed', 'AppleWalkingSteadiness', 'RunningStrideLength', 'RunningVerticalOscillation', 'RunningGroundContactTime', 'HeartRateRecoveryOneMinute', 'RunningPower', 'RunningSpeed', 'SleepAnalysis', 'AppleStandHour', 'LowHeartRateEvent', 'HeartRateVariabilitySDNN']


In [733]:

personal_records = ['Height', 
                    'BodyMass',
                    'RestingHeartRate',
                    'WalkingHeartRateAverage',
                    'AppleWalkingSteadiness',
                    'SixMinuteWalkTestDistance']

basis_records = ['HeartRate', 
                 'OxygenSaturation', 
                 'RespiratoryRate', 
                 'StepCount', 
                 'FlightsClimbed', 
                 'AppleExerciseTime',
                 'VO2Max',
                 'EnvironmentalAudioExposure', 
                 'HeadphoneAudioExposure',
                 'WalkingDoubleSupportPercentage', 
                 'AppleStandTime', 
                 'WalkingSpeed', 
                 'WalkingStepLength', 
                 'WalkingAsymmetryPercentage',
                 'StairAscentSpeed', 
                 'StairDescentSpeed',
                 'SleepAnalysis', 
                 'AppleStandHour', 
                 'LowHeartRateEvent', 
                 'HeartRateVariabilitySDNN',
                 'HeartRateRecoveryOneMinute']

workout_records = ['DistanceWalkingRunning', 
                   'BasalEnergyBurned', 
                   'ActiveEnergyBurned', 
                   'DistanceCycling', 
                   'DistanceSwimming', 
                   'SwimmingStrokeCount',
                   'DistanceDownhillSnowSports',
                   'RunningStrideLength', 
                   'RunningVerticalOscillation', 
                   'RunningGroundContactTime', 
                   'RunningPower',
                   'RunningSpeed']


##### Personal Records

In [750]:
# dictionary of DataFrames for filtered 'record_data'
personal_dict = {}

# create new DataFrame for every interested data
for type in personal_records:
   print(type)
   personal_dict[type] = df_records[df_records['type'] == type].rename(columns={"value": type}).sort_values(by='Date')

   # create from/to-date columns --> drop irrelevant columns
   personal_dict[type]['fromDate'] = personal_dict[type]['Date']
   personal_dict[type]['toDate'] = personal_dict[type]['Date'].shift(-1)
   personal_dict[type] = personal_dict[type].drop(['creationDate', 'startDate', 'endDate', 'Date', 'startTime', 'endTime'], axis = 1)

   #display(personal_dict[type].tail(5))

Height
BodyMass
RestingHeartRate
WalkingHeartRateAverage
AppleWalkingSteadiness
SixMinuteWalkTestDistance


In [751]:
personal_record_df = pd.DataFrame()

for type in personal_records:
    frames = [personal_record_df, personal_dict[type]]
    personal_record_df = pd.concat(frames)

In [752]:
FactPersonalRecords = pd.melt(personal_record_df,
                      id_vars=['fromDate',
                               'toDate',
                               'unit'], 
                     value_vars=personal_records, 
                     var_name='type',
                     value_name='value')

FactPersonalRecords = FactPersonalRecords[FactPersonalRecords['value'].notna()]
FactPersonalRecords = FactPersonalRecords[['type', 'value', 'unit', 'fromDate', 'toDate']]

In [753]:
FactPersonalRecords

,type,value,unit,fromDate,toDate
0,Height,181.0,cm,2022-03-28,2022-04-30
1,Height,181.0,cm,2022-04-30,None
1238,BodyMass,80.9,kg,2017-06-11,2017-06-28
1239,BodyMass,80.9,kg,2017-06-28,2018-02-07
1240,BodyMass,82.7,kg,2018-02-07,2018-03-11
...,...,...,...,...,...
7411,SixMinuteWalkTestDistance,500.0,m,2023-08-19,2023-08-27
7412,SixMinuteWalkTestDistance,500.0,m,2023-08-27,2023-09-03
7413,SixMinuteWalkTestDistance,500.0,m,2023-09-03,2023-09-10
7414,SixMinuteWalkTestDistance,500.0,m,2023-09-10,2023-09-17


##### Basis Records

In [754]:
# dictionary of DataFrames for filtered 'record_data'
basis_dict = {}

# create new DataFrame for every interested data
for type in basis_records:
   print(type)
   basis_dict[type] = df_records[df_records['type'] == type].rename(columns={"value": type}).sort_values(by='Date')

   #display(basis_dict[type].tail(3))

HeartRate
OxygenSaturation
RespiratoryRate
StepCount
FlightsClimbed
AppleExerciseTime
VO2Max
EnvironmentalAudioExposure
HeadphoneAudioExposure
WalkingDoubleSupportPercentage
AppleStandTime
WalkingSpeed
WalkingStepLength
WalkingAsymmetryPercentage
StairAscentSpeed
StairDescentSpeed
SleepAnalysis
AppleStandHour
LowHeartRateEvent
HeartRateVariabilitySDNN
HeartRateRecoveryOneMinute


In [755]:
basis_record_df = pd.DataFrame()

for type in basis_records:
    frames = [basis_record_df, basis_dict[type]]
    basis_record_df = pd.concat(frames)

basis_record_df = basis_record_df.drop(['creationDate', 'startTime', 'endTime'], axis = 1)

In [741]:
FactBasisRecords = pd.melt(basis_record_df,
                      id_vars=['unit',
                               'startDate',
                               'endDate',
                               'Date'], 
                     value_vars=basis_records, 
                     var_name='type',
                     value_name='value')

FactBasisRecords = FactBasisRecords[FactBasisRecords['value'].notna()]
FactBasisRecords = FactBasisRecords[['type', 'value', 'unit', 'Date', 'startDate', 'endDate']]

In [766]:
FactBasisRecords['startDate'] = pd.to_datetime(FactBasisRecords['startDate'])
FactBasisRecords['endDate'] = pd.to_datetime(FactBasisRecords['endDate'])
FactBasisRecords['Date'] = pd.to_datetime(FactBasisRecords['Date'])


# Create dataframe that lists workout Starts/Ends
wo_start_end = DimWorkouts[['workout_pk', 'Date', 'startDate', 'endDate']].drop_duplicates()
wo_start_end['startDate'] = pd.to_datetime(wo_start_end['startDate'])
wo_start_end['endDate'] = pd.to_datetime(wo_start_end['endDate'])
wo_start_end['Date'] = pd.to_datetime(wo_start_end['Date'])

FactBasisRecordsMerged = pd.merge(FactBasisRecords, wo_start_end, on = 'Date', how='left', suffixes=('_records', '_wo'))
FactBasisRecordsMerged['workout_pk'] = FactBasisRecordsMerged['workout_pk'].astype('Int64')

FactBasisRecordsMerged['isWO'] = np.where(
    (FactBasisRecordsMerged['startDate_records'] >= FactBasisRecordsMerged['startDate_wo']) & 
    (FactBasisRecordsMerged['startDate_records'] <= FactBasisRecordsMerged['endDate_wo']), 1, 0)

FactBasisRecords = FactBasisRecordsMerged.drop(['startDate_wo', 'endDate_wo'], axis=1)

In [776]:
############### INVESTIGATE WHETHER WORKOUT_PK IS SET CORRECTLY if ISWO == 0!!!!! 
############### INVESTIGATE WHETHER WORKOUT_PK IS SET CORRECTLY if ISWO == 0!!!!! 
############### INVESTIGATE WHETHER WORKOUT_PK IS SET CORRECTLY if ISWO == 0!!!!! 

In [777]:
FactWorkoutRecords

,type,value,unit,Date,startDate_records,endDate_records,workout_pk,isWO
0,DistanceWalkingRunning,0.484882,km,2018-10-13,2018-10-13 11:10:17+02:00,2018-10-13 11:19:40+02:00,<NA>,0
1,DistanceWalkingRunning,3.251390,km,2018-10-13,2018-10-13 11:19:40+02:00,2018-10-13 13:43:55+02:00,<NA>,0
2,DistanceWalkingRunning,0.064720,km,2018-10-13,2018-10-13 13:43:55+02:00,2018-10-13 13:48:07+02:00,<NA>,0
3,DistanceWalkingRunning,0.012780,km,2018-10-13,2018-10-13 13:58:35+02:00,2018-10-13 13:58:38+02:00,<NA>,0
4,DistanceWalkingRunning,0.257880,km,2018-10-13,2018-10-13 15:18:07+02:00,2018-10-13 15:27:36+02:00,<NA>,0
...,...,...,...,...,...,...,...,...
1904654,RunningSpeed,13.124000,km/hr,2023-09-19,2023-09-19 12:25:47+02:00,2023-09-19 12:25:47+02:00,571,1
1904655,RunningSpeed,13.236400,km/hr,2023-09-19,2023-09-19 12:25:50+02:00,2023-09-19 12:25:50+02:00,571,1
1904656,RunningSpeed,13.307800,km/hr,2023-09-19,2023-09-19 12:25:53+02:00,2023-09-19 12:25:53+02:00,571,1
1904657,RunningSpeed,13.049100,km/hr,2023-09-19,2023-09-19 12:25:58+02:00,2023-09-19 12:25:58+02:00,571,1


##### Workout Records

In [757]:
# dictionary of DataFrames for filtered 'record_data'
workout_dict = {}

# create new DataFrame for every interested data
for type in workout_records:
   print(type)
   workout_dict[type] = df_records[df_records['type'] == type].rename(columns={"value": type}).sort_values(by='Date')

   #display(workout_dict[type].tail(3))

DistanceWalkingRunning
BasalEnergyBurned
ActiveEnergyBurned
DistanceCycling
DistanceSwimming
SwimmingStrokeCount
DistanceDownhillSnowSports
RunningStrideLength
RunningVerticalOscillation
RunningGroundContactTime
RunningPower
RunningSpeed


In [758]:
workout_record_df = pd.DataFrame()

for type in workout_records:
    frames = [workout_record_df, workout_dict[type]]
    workout_record_df = pd.concat(frames)

workout_record_df = workout_record_df.drop(['creationDate', 'startTime', 'endTime'], axis = 1)

In [759]:
FactWorkoutRecords = pd.melt(workout_record_df,
                      id_vars=['unit',
                               'startDate',
                               'endDate',
                               'Date'], 
                     value_vars=workout_records, 
                     var_name='type',
                     value_name='value')

FactWorkoutRecords = FactWorkoutRecords[FactWorkoutRecords['value'].notna()]
FactWorkoutRecords = FactWorkoutRecords[['type', 'value', 'unit', 'Date', 'startDate', 'endDate']]

In [774]:
FactWorkoutRecords['startDate'] = pd.to_datetime(FactWorkoutRecords['startDate'])
FactWorkoutRecords['endDate'] = pd.to_datetime(FactWorkoutRecords['endDate'])
FactWorkoutRecords['Date'] = pd.to_datetime(FactWorkoutRecords['Date'])


# Create dataframe that lists workout Starts/Ends
wo_start_end = DimWorkouts[['workout_pk', 'Date', 'startDate', 'endDate']].drop_duplicates()
wo_start_end['startDate'] = pd.to_datetime(wo_start_end['startDate'])
wo_start_end['endDate'] = pd.to_datetime(wo_start_end['endDate'])
wo_start_end['Date'] = pd.to_datetime(wo_start_end['Date'])

FactWorkoutRecordsMerged = pd.merge(FactWorkoutRecords, wo_start_end, on = 'Date', how='left', suffixes=('_records', '_wo'))
FactWorkoutRecordsMerged['workout_pk'] = FactWorkoutRecordsMerged['workout_pk'].astype('Int64')

FactWorkoutRecordsMerged['isWO'] = np.where(
    (FactWorkoutRecordsMerged['startDate_records'] >= FactWorkoutRecordsMerged['startDate_wo']) & 
    (FactWorkoutRecordsMerged['startDate_records'] <= FactWorkoutRecordsMerged['endDate_wo']), 1, 0)

FactWorkoutRecords = FactWorkoutRecordsMerged.drop(['startDate_wo', 'endDate_wo'], axis=1)

In [775]:
FactWorkoutRecords

,type,value,unit,Date,startDate_records,endDate_records,workout_pk,isWO
0,DistanceWalkingRunning,0.484882,km,2018-10-13,2018-10-13 11:10:17+02:00,2018-10-13 11:19:40+02:00,<NA>,0
1,DistanceWalkingRunning,3.251390,km,2018-10-13,2018-10-13 11:19:40+02:00,2018-10-13 13:43:55+02:00,<NA>,0
2,DistanceWalkingRunning,0.064720,km,2018-10-13,2018-10-13 13:43:55+02:00,2018-10-13 13:48:07+02:00,<NA>,0
3,DistanceWalkingRunning,0.012780,km,2018-10-13,2018-10-13 13:58:35+02:00,2018-10-13 13:58:38+02:00,<NA>,0
4,DistanceWalkingRunning,0.257880,km,2018-10-13,2018-10-13 15:18:07+02:00,2018-10-13 15:27:36+02:00,<NA>,0
...,...,...,...,...,...,...,...,...
1904654,RunningSpeed,13.124000,km/hr,2023-09-19,2023-09-19 12:25:47+02:00,2023-09-19 12:25:47+02:00,571,1
1904655,RunningSpeed,13.236400,km/hr,2023-09-19,2023-09-19 12:25:50+02:00,2023-09-19 12:25:50+02:00,571,1
1904656,RunningSpeed,13.307800,km/hr,2023-09-19,2023-09-19 12:25:53+02:00,2023-09-19 12:25:53+02:00,571,1
1904657,RunningSpeed,13.049100,km/hr,2023-09-19,2023-09-19 12:25:58+02:00,2023-09-19 12:25:58+02:00,571,1


In [ ]:
############### INVESTIGATE WHETHER WORKOUT_PK IS SET CORRECTLY if ISWO == 0!!!!! 
############### INVESTIGATE WHETHER WORKOUT_PK IS SET CORRECTLY if ISWO == 0!!!!! 
############### INVESTIGATE WHETHER WORKOUT_PK IS SET CORRECTLY if ISWO == 0!!!!! 

In [ ]:
#################################################################################################################

##### Investigate Workout Metrics

record_df = pd.DataFrame()

for type in record_types:
    frames = [record_df, record_dict[type]]
    record_df = pd.concat(frames)

record_df['Date'] = pd.to_datetime(record_df['startDate']).dt.strftime('%Y-%m-%d')
record_df['startDate'] = pd.to_datetime(record_df['startDate'])

record_df['Date'] = pd.to_datetime(record_df['startDate']).dt.strftime('%Y-%m-%d')
record_df['startDate'] = pd.to_datetime(record_df['startDate'])


# Create dataframe that lists workout Starts/Ends
wo_start_end = DimWorkouts[['workout_pk', 'Date', 'startDate', 'endDate']].drop_duplicates()

merged = pd.merge(record_df, wo_start_end, on = 'Date', how='left', suffixes=('_records', '_wo'))
merged['workout_pk'] = merged['workout_pk'].astype('Int64')

merged['isWO'] = np.where(
    (merged['startDate_records'] >= merged['startDate_wo']) & 
    (merged['startDate_records'] <= merged['endDate_wo']), 1, 0)

records = merged.drop(['startDate_wo', 'endDate_wo'], axis=1)

records = merged.drop(['startDate_wo', 'endDate_wo'], axis=1)

records

records_types = list(records['type'].unique())

FactRecords = pd.melt(records,
                      id_vars=['Date',
                               'creationDate', 
                               'startDate_records',
                               'startTime', 
                               'endDate_records',
                               'endTime',
                               'isWO',
                               'workout_pk',
                               'unit'], 
                     value_vars=records_types, 
                     var_name='type',
                     value_name='value').dropna()

### Workout Routes

In [396]:
# Path to the XML file exported form Apple Health app
XML_routes = "C:/PythonProjects/AppleHealth/input_files/apple_health_export/workout-routes"

In [397]:
example_route = list(os.listdir(XML_routes))[0]

tree = ET.parse(f'{XML_routes}/{example_route}')
root = tree.getroot()

In [398]:
attributes = []
for child in root:
    attributes.append(child.tag)

attributes = list(set(attributes))
print(attributes)

['{http://www.topografix.com/GPX/1/1}trk', '{http://www.topografix.com/GPX/1/1}metadata']


In [399]:
level_1_list = []
level_2_list = []
level_3_list = []
level_4_list = []
level_5_list = []

a_list = list(root.iter('{http://www.topografix.com/GPX/1/1}trk'))
for a in range(len(a_list)):
    b_list = list(a_list[a])
    
    for b in range(len(b_list)):
        level_1_list.append(b_list[b].tag)
        c_list = list(b_list[b])

        for c in range(len(c_list)):
            level_2_list.append(c_list[c].tag)
            d_list = list(c_list[c])

            for d in range(len(d_list)):
                level_3_list.append(d_list[d].tag)
                e_list = list(d_list[d])

                for e in range(len(e_list)):
                    level_4_list.append(e_list[e].tag)
                    f_list = list(e_list[e])

                    for f in range(len(f_list)):
                        level_5_list.append(f_list[f].tag)

level_1_list = list(set(level_1_list))
level_2_list = list(set(level_2_list))
level_3_list = list(set(level_3_list))
level_4_list = list(set(level_4_list))
level_5_list = list(set(level_5_list))

In [400]:
### EMPTY LIST: no further levels to investigate
print(level_1_list)
print(level_2_list)
print(level_3_list)
print(level_4_list)
print(level_5_list)

concat_list = list(set(level_1_list + level_2_list + level_3_list + level_4_list))

['{http://www.topografix.com/GPX/1/1}name', '{http://www.topografix.com/GPX/1/1}trkseg']
['{http://www.topografix.com/GPX/1/1}trkpt']
['{http://www.topografix.com/GPX/1/1}ele', '{http://www.topografix.com/GPX/1/1}time', '{http://www.topografix.com/GPX/1/1}extensions']
['{http://www.topografix.com/GPX/1/1}course', '{http://www.topografix.com/GPX/1/1}speed', '{http://www.topografix.com/GPX/1/1}hAcc', '{http://www.topografix.com/GPX/1/1}vAcc']
[]


In [401]:
route_metrics = ['trkpt', 'ele', 'time', 'course', 'speed', 'hAcc', 'vAcc']

In [402]:
combined_routes_df = pd.DataFrame()

for file in list(os.listdir(XML_routes)):
    route_df = pd.DataFrame()
    tree = ET.parse(f'{XML_routes}/{file}')

    for metric in route_metrics:
        # create metric lists
        globals()[f"list_{metric}"] = []
        source = "{http://www.topografix.com/GPX/1/1}"+str(metric)

        # loop elements of metric
        for elem in tree.findall(f".//{source}"):

            # add elements to metric lists (trkpt's have different format)
            if elem.tag == '{http://www.topografix.com/GPX/1/1}trkpt':
                globals()[f"list_{metric}"].append(elem.attrib)
            else:
                globals()[f"list_{metric}"].append(elem.text)

        # check whether column is time (TIME HAS ALWAYS N+1 ENTRIES --> ignore first entry) 
        if metric == 'time':
            route_df[metric] = globals()[f"list_{metric}"][1:]
        else:
            route_df[metric] = globals()[f"list_{metric}"]

    # add route name as column
    route_df['route_name'] = file
    # union "route" dataframe to overall "combined_routes" dataframe
    combined_routes_df = pd.concat([combined_routes_df, route_df])    

print(f"{len(list(combined_routes_df['route_name'].unique()))} routes added to dataframe!")

325 routes added to dataframe!


In [403]:
DimWorkouts['route_name'] = DimWorkouts['path'].str.split(pat = '/')
DimWorkouts['route_name'] = DimWorkouts['route_name'].str[-1]
DimRoutes = pd.merge(combined_routes_df, DimWorkouts[['route_name', 'workout_pk']], on = 'route_name', how = 'left')

### Save Tables

In [ ]:
FactRecords.to_parquet('./output_files/frontend_files/FactRecords.parquet.gzip', compression='gzip')
DimWorkouts.to_parquet('./output_files/frontend_files/DimWorkouts.parquet.gzip', compression='gzip')
DimRoutes.to_parquet('./output_files/frontend_files/DimRoutes.parquet.gzip', compression='gzip')